In [ ]:
# === Google Colab & Kaggle Setup ===
# Install the Kaggle package and upload your kaggle.json credentials.
!pip install kaggle

from google.colab import files
# Upload your kaggle.json file when prompted.
files.upload()

# Move the kaggle.json file to the correct location and set proper permissions.
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# === Download and Unzip Datasets from Kaggle ===
# Replace 'your_username/div2k-dataset' and 'your_username/flickr2k' with the correct dataset identifiers.
!kaggle datasets download -d joe1995/div2k-dataset
!unzip -q div2k-dataset.zip -d /content/div2k

!kaggle datasets download -d hliang001/flickr2k
!unzip -q flickr2k.zip -d /content/flickr2k

Saving kaggle.json to kaggle.json
Dataset URL: https://www.kaggle.com/datasets/joe1995/div2k-dataset
License(s): unknown
100% 3.71G/3.71G [03:05<00:00, 24.5MB/s]
100% 3.71G/3.71G [03:06<00:00, 21.4MB/s]
Dataset URL: https://www.kaggle.com/datasets/hliang001/flickr2k
License(s): unknown
100% 20.0G/20.0G [16:57<00:00, 23.9MB/s]
100% 20.0G/20.0G [16:57<00:00, 21.1MB/s]


In [ ]:
import os
import glob
import random
import math
from PIL import Image
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split
import torchvision.transforms as transforms
import torchvision.transforms.functional as TF
from tqdm import tqdm

#######################################
# 1. Dataset Definition (Same as before)
#######################################

class SRDataset(Dataset):
    """
    A dataset for single image super-resolution.
    It loads HR images from multiple directories, applies augmentation,
    and generates a corresponding LR image via bicubic downsampling.
    """
    def __init__(self, image_dirs, crop_size=96, scale=4, mode='train'):
        self.image_paths = []
        for d in image_dirs:
            self.image_paths.extend(glob.glob(os.path.join(d, "*.png")))
            self.image_paths.extend(glob.glob(os.path.join(d, "*.jpg")))
            self.image_paths.extend(glob.glob(os.path.join(d, "*.jpeg")))
        self.crop_size = crop_size
        self.scale = scale
        self.mode = mode

        self.to_tensor = transforms.ToTensor()
        if mode == 'train':
            self.augment = transforms.Compose([
                transforms.RandomCrop(crop_size),
                transforms.RandomHorizontalFlip(),
                transforms.RandomChoice([
                    transforms.Lambda(lambda img: TF.rotate(img, 0)),
                    transforms.Lambda(lambda img: TF.rotate(img, 90)),
                    transforms.Lambda(lambda img: TF.rotate(img, 180)),
                    transforms.Lambda(lambda img: TF.rotate(img, 270))
                ]),
            ])
        else:
            self.augment = transforms.CenterCrop(crop_size)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        hr_image = Image.open(img_path).convert("RGB")
        hr_image = self.augment(hr_image)
        lr_size = (hr_image.width // self.scale, hr_image.height // self.scale)
        lr_image = hr_image.resize(lr_size, resample=Image.BICUBIC)
        hr_tensor = self.to_tensor(hr_image)
        lr_tensor = self.to_tensor(lr_image)
        return lr_tensor, hr_tensor

#######################################
# 2. PSNR Calculation
#######################################

def calculate_psnr(sr, hr, max_val=1.0):
    mse = F.mse_loss(sr, hr)
    if mse == 0:
        return 100.0
    psnr = 10 * torch.log10((max_val ** 2) / mse)
    return psnr.item()

#######################################
# 3. Charbonnier Loss
#######################################

class CharbonnierLoss(nn.Module):
    """
    Charbonnier loss: a robust variant of L1 loss.
      loss(x, y) = mean(sqrt((x - y)^2 + eps^2))
    """
    def __init__(self, eps=1e-3):
        super(CharbonnierLoss, self).__init__()
        self.eps = eps

    def forward(self, x, y):
        return torch.mean(torch.sqrt((x - y)**2 + self.eps**2))

#######################################
# 4. RGT-S Generator Architecture (Improved)
#######################################

class RGTransformerBlock(nn.Module):
    """
    An improved recursive-generalization transformer block.
    It flattens spatial features into tokens, applies global self-attention
    followed by a local self-attention branch, then an MLP.
    """
    def __init__(self, dim, num_heads, mlp_ratio=4.0, dropout=0.1):
        super(RGTransformerBlock, self).__init__()
        self.norm1 = nn.LayerNorm(dim)
        self.attn_global = nn.MultiheadAttention(embed_dim=dim, num_heads=num_heads, dropout=dropout)
        self.norm2 = nn.LayerNorm(dim)
        self.attn_local = nn.MultiheadAttention(embed_dim=dim, num_heads=num_heads, dropout=dropout)
        self.norm3 = nn.LayerNorm(dim)
        self.mlp = nn.Sequential(
            nn.Linear(dim, int(dim * mlp_ratio)),
            nn.GELU(),
            nn.Linear(int(dim * mlp_ratio), dim),
            nn.Dropout(dropout)
        )

    def forward(self, x):
        # x: (B, C, H, W). Flatten spatial dims to (N, B, C) where N=H*W.
        B, C, H, W = x.shape
        N = H * W
        x_flat = x.view(B, C, N).permute(2, 0, 1)  # shape: (N, B, C)

        # Global self-attention
        x_norm = self.norm1(x_flat)
        attn_global, _ = self.attn_global(x_norm, x_norm, x_norm)
        x_global = x_flat + attn_global

        # Local self-attention (could use windowing; here we use full tokens for simplicity)
        x_norm2 = self.norm2(x_global)
        attn_local, _ = self.attn_local(x_norm2, x_norm2, x_norm2)
        x_local = x_global + attn_local

        # MLP block
        x_norm3 = self.norm3(x_local)
        mlp_out = self.mlp(x_norm3)
        x_out = x_local + mlp_out

        # Reshape back to (B, C, H, W)
        x_out = x_out.permute(1, 2, 0).view(B, C, H, W)
        return x_out

class RGT_S_Generator(nn.Module):
    """
    Improved RGT-S Generator for 4x image super-resolution.
    Architecture:
      - Shallow convolutional embedding.
      - A stack of transformer blocks (RGTransformerBlock) with increased capacity.
      - A convolutional trunk (plus an extra refinement conv) with a skip connection.
      - Upsampling via PixelShuffle.
      - Final convolution to output the image.
    """
    def __init__(self, in_channels=3, out_channels=3, embed_dim=128, depth=12, num_heads=8, mlp_ratio=4.0, scale=4, dropout=0.1):
        super(RGT_S_Generator, self).__init__()
        self.conv_first = nn.Conv2d(in_channels, embed_dim, kernel_size=3, padding=1)
        self.transformer_blocks = nn.Sequential(*[
            RGTransformerBlock(dim=embed_dim, num_heads=num_heads, mlp_ratio=mlp_ratio, dropout=dropout)
            for _ in range(depth)
        ])
        # Convolutional trunk with extra refinement
        self.trunk_conv = nn.Conv2d(embed_dim, embed_dim, kernel_size=3, padding=1)
        self.refine_conv = nn.Conv2d(embed_dim, embed_dim, kernel_size=3, padding=1)

        # Upsampling module: for 4x scaling, use two PixelShuffle blocks.
        upsample_layers = []
        num_upsample = int(math.log(scale, 2))
        for _ in range(num_upsample):
            upsample_layers += [
                nn.Conv2d(embed_dim, embed_dim * 4, kernel_size=3, padding=1),
                nn.PixelShuffle(2),
                nn.GELU()
            ]
        self.upsampling = nn.Sequential(*upsample_layers)
        self.conv_last = nn.Conv2d(embed_dim, out_channels, kernel_size=3, padding=1)

    def forward(self, x):
        fea = self.conv_first(x)
        trans_out = self.transformer_blocks(fea)
        trunk = self.trunk_conv(trans_out)
        refined = self.refine_conv(trunk)
        fea = fea + refined  # global skip connection
        out = self.upsampling(fea)
        out = self.conv_last(out)
        return out

#######################################
# 5. Training Loop (Optimized for High PSNR)
#######################################

def train_rgt_s(generator, train_loader, val_loader, num_epochs, optimizer, scheduler, device):
    best_val_psnr = 0.0
    criterion = CharbonnierLoss().to(device)

    for epoch in range(1, num_epochs + 1):
        generator.train()
        train_psnr_total = 0.0
        train_batches = 0
        epoch_loss = 0.0

        pbar = tqdm(train_loader, desc=f"Epoch {epoch} Train")
        for lr_imgs, hr_imgs in pbar:
            lr_imgs, hr_imgs = lr_imgs.to(device), hr_imgs.to(device)
            optimizer.zero_grad()
            sr_imgs = generator(lr_imgs)
            loss = criterion(sr_imgs, hr_imgs)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(generator.parameters(), max_norm=1.0)
            optimizer.step()

            batch_psnr = calculate_psnr(sr_imgs, hr_imgs)
            train_psnr_total += batch_psnr
            train_batches += 1
            epoch_loss += loss.item()
            pbar.set_postfix(loss=loss.item(), PSNR=batch_psnr)

        avg_train_psnr = train_psnr_total / train_batches

        # Validation phase
        generator.eval()
        val_psnr_total = 0.0
        val_batches = 0
        with torch.no_grad():
            pbar_val = tqdm(val_loader, desc=f"Epoch {epoch} Val")
            for lr_imgs, hr_imgs in pbar_val:
                lr_imgs, hr_imgs = lr_imgs.to(device), hr_imgs.to(device)
                sr_imgs = generator(lr_imgs)
                batch_psnr = calculate_psnr(sr_imgs, hr_imgs)
                val_psnr_total += batch_psnr
                val_batches += 1
                pbar_val.set_postfix(PSNR=batch_psnr)
        avg_val_psnr = val_psnr_total / val_batches

        scheduler.step()

        print(f"Epoch {epoch}: Loss: {epoch_loss/len(train_loader):.4f}, Train PSNR: {avg_train_psnr:.2f} dB, Val PSNR: {avg_val_psnr:.2f} dB")

        if avg_val_psnr > best_val_psnr:
            best_val_psnr = avg_val_psnr
            torch.save(generator.state_dict(), "best_rgt_s_generator.pth")
            print(f"--> Saved new best model with Val PSNR: {best_val_psnr:.2f} dB")

#######################################
# 6. Main Function
#######################################

def main():
    # Update directories to point to the extracted folders in Colab.
    div2k_dir = "/content/div2k/DIV2K_train_HR/DIV2K_train_HR"
    flickr2k_dir = "/content/flickr2k/Flickr2K_HR"
    image_dirs = [div2k_dir, flickr2k_dir]

    crop_size = 96   # HR crop size
    scale = 4
    full_dataset = SRDataset(image_dirs=image_dirs, crop_size=crop_size, scale=scale, mode='train')
    total_size = len(full_dataset)
    train_size = int(0.9 * total_size)
    val_size = total_size - train_size
    train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

    train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=4, pin_memory=True)
    val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False, num_workers=4, pin_memory=True)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    # Initialize RGT-S generator with increased capacity
    generator = RGT_S_Generator(scale=scale, embed_dim=128, depth=12, num_heads=8, mlp_ratio=4.0, dropout=0.1).to(device)

    optimizer = optim.Adam(generator.parameters(), lr=1e-4, weight_decay=1e-4)
    num_epochs = 200
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs)

    train_rgt_s(generator, train_loader, val_loader, num_epochs, optimizer, scheduler, device)

if __name__ == "__main__":
    main()


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
Epoch 1 Val: 100%|██████████| 5/5 [00:06<00:00,  1.36s/it, PSNR=16.4]


Epoch 1: Loss: 0.1736, Train PSNR: 13.27 dB, Val PSNR: 16.36 dB
--> Saved new best model with Val PSNR: 16.36 dB


Epoch 2 Val: 100%|██████████| 5/5 [00:06<00:00,  1.32s/it, PSNR=17.2]


Epoch 2: Loss: 0.1088, Train PSNR: 16.81 dB, Val PSNR: 17.64 dB
--> Saved new best model with Val PSNR: 17.64 dB


Epoch 3 Val: 100%|██████████| 5/5 [00:06<00:00,  1.34s/it, PSNR=20.7]


...

Epoch 85: Loss: 0.0304, Train PSNR: 25.25 dB, Val PSNR: 25.87 dB


Epoch 86 Val: 100%|██████████| 5/5 [00:06<00:00,  1.21s/it, PSNR=24.9]


Epoch 86: Loss: 0.0299, Train PSNR: 25.40 dB, Val PSNR: 26.76 dB


Epoch 87 Train:  71%|███████   | 32/45 [00:43<00:17,  1.36s/it, PSNR=25.6, loss=0.0295]


KeyboardInterrupt: 